### Baseline модель для определения именованных сущностей по кейсу от Rutube.
Поскольку нам нужно распознать нестандартные NER, можно воспользоваться помощью языковых моделей, в данном случае - Bert.
Данные вы уже получили  - это разметка, сделанная на Толоке, она не идеальна, но это часть практической задачи, с которой можно столкнуться в реальности. 

Небольшое введение в NER https://habr.com/ru/companies/contentai/articles/449514/

In [ ]:
#!pip install transformers[torch] 
#!pip install datasets seqeval
#!pip install corus razdel 
#!pip install torch==2.0.0
#!pip install pandas

In [57]:
# считаем данные
import os 
import torch
import pandas as pd

batch_size = 32
num_train_epochs = 25
model_checkpoint = "cointegrated/rubert-tiny"
name_folder_and_model = 'ner_bert_25.bin'

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
is_cuda = torch.cuda.is_available()
print(is_cuda)
data = pd.read_csv("ner_data_train.csv")
datatest = pd.read_csv("ner_data_test.csv")


True


In [58]:
# данные спарсены с Толоки, поэтому могут иметь проблемы с символами и их нужно избежать, 
# удалить лишние '\' например, преобразовать из str в список dict-ов
import json
df = data.copy()
df['entities'] = df['entities'].apply(lambda l: l.replace('\,', ',')if isinstance(l, str) else l)
df['entities'] = df['entities'].apply(lambda l: l.replace('\\\\', '\\')if isinstance(l, str) else l)
df['entities'] = df['entities'].apply(lambda l: '[' + l + ']'if isinstance(l, str) else l)
df['entities'] = df['entities'].apply(lambda l: json.loads(l)if isinstance(l, str) else l)

dft = datatest.copy()
dft['entities_prediction'] = dft['entities_prediction'].apply(lambda l: l.replace('\,', ',')if isinstance(l, str) else l)
dft['entities_prediction'] = dft['entities_prediction'].apply(lambda l: l.replace('\\\\', '\\')if isinstance(l, str) else l)
dft['entities_prediction'] = dft['entities_prediction'].apply(lambda l: '[' + l + ']'if isinstance(l, str) else l)
dft['entities_prediction'] = dft['entities_prediction'].apply(lambda l: json.loads(l)if isinstance(l, str) else l)

#### Оригинал туториала на медицинских данных можно посмотреть тут https://gist.github.com/avidale/cacf235aebeaaf4c578389e1146c3c57

In [59]:
# Теперь из наших данных нам нужно извлечь для каждого слова (токена) его тег (label) из разметки, чтобы потом предать в модель классификации токенов
from razdel import tokenize

def extract_labels(item):
    
    # воспользуемся удобным токенайзером из библиотеки razdel, 
    # она помимо разбиения на слова, сохраняет важные для нас числа - начало и конец слова в токенах
    
    raw_toks = list(tokenize(item['video_info']))
    words = [tok.text for tok in raw_toks]
    # присвоим для начала каждому слову тег 'О' - тег, означающий отсутствие NER-а
    word_labels = ['O'] * len(raw_toks)
    char2word = [None] * len(item['video_info'])
    # так как NER можем состаять из нескольких слов, то нам нужно сохранить эту инфорцию
    for i, word in enumerate(raw_toks):
        char2word[word.start:word.stop] = [i] * len(word.text)

    labels = item['entities']
    if isinstance(labels, dict):
        labels = [labels]
    if labels is not None:
        for e in labels:
            if e['label'] != 'не найдено':
                e_words = sorted({idx for idx in char2word[e['offset']:e['offset']+e['length']] if idx is not None})
                if e_words:
                    word_labels[e_words[0]] = 'B-' + e['label']
                    for idx in e_words[1:]:
                        word_labels[idx] = 'I-' + e['label']
                else:
                    continue
            else:
                continue
        return {'tokens': words, 'tags': word_labels}
    else: return {'tokens': words, 'tags': word_labels}


### Обработаем датасет и разобьем на трейн и тест

In [60]:
from sklearn.model_selection import train_test_split
ner_data = [extract_labels(item) for i, item in df.iterrows()]


In [61]:
import numpy as np
def extract_labels_test(item):
    
    # воспользуемся удобным токенайзером из библиотеки razdel, 
    # она помимо разбиения на слова, сохраняет важные для нас числа - начало и конец слова в токенах
    
    raw_toks = list(tokenize(item['video_info']))
    words = [tok.text for tok in raw_toks]
    # присвоим для начала каждому слову тег 'О' - тег, означающий отсутствие NER-а
    word_labels = ['O'] * len(raw_toks)
    char2word = [None] * len(item['video_info'])
    # так как NER можем состаять из нескольких слов, то нам нужно сохранить эту инфорцию
    for i, word in enumerate(raw_toks):
        char2word[word.start:word.stop] = [i] * len(word.text)

    labels = item['entities_prediction']
    if isinstance(labels, dict):
        labels = [labels]
    if not pd.isna(labels):
        for e in labels:
            if e['label'] != 'не найдено':
                e_words = sorted({idx for idx in char2word[e['offset']:e['offset']+e['length']] if idx is not None})
                if e_words:
                    word_labels[e_words[0]] = 'B-' + e['label']
                    for idx in e_words[1:]:
                        word_labels[idx] = 'I-' + e['label']
                else:
                    continue
            else:
                continue
        return {'tokens': words, 'tags': word_labels}
    else: return {'tokens': words, 'tags': word_labels}
ner_data_test = [extract_labels_test(item) for i, item in dft.iterrows()]


In [62]:

ner_train, ner_test = train_test_split(ner_data, test_size=0.1, random_state=1)

In [63]:
pd.options.display.max_colwidth = 300

In [64]:
len(ner_data_test)

1606

#### Посмотрим на получившиеся теги
Подробнее почитать про BIO теги можно тут https://datascience.stackexchange.com/questions/63399/what-is-bio-tags-for-creating-custom-ner-named-entity-recognization

In [65]:
#ner_train = ner_data
#ner_test = ner_data_test

In [66]:
label_list = sorted({label for item in ner_train for label in item['tags']})
if 'O' in label_list:
    label_list.remove('O')
    label_list = ['O'] + label_list
label_list

['O',
 'B-Дата',
 'B-бренд',
 'B-вид спорта',
 'B-видеоигра',
 'B-команда',
 'B-лига',
 'B-локация',
 'B-модель',
 'B-название проекта',
 'B-организация',
 'B-персона',
 'B-сезон',
 'B-серия',
 'I-Дата',
 'I-бренд',
 'I-вид спорта',
 'I-видеоигра',
 'I-команда',
 'I-лига',
 'I-локация',
 'I-модель',
 'I-название проекта',
 'I-организация',
 'I-персона',
 'I-сезон',
 'I-серия']

In [67]:
from datasets import Dataset, DatasetDict

In [68]:
ner_data = DatasetDict({
    'train': Dataset.from_pandas(pd.DataFrame(ner_train)),
    'test': Dataset.from_pandas(pd.DataFrame(ner_test))
})
ner_data

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 5779
    })
    test: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 643
    })
})

### Запустим модель RuBert-tiny - классический Bert, поверх которого навешен слой классификации токенов.

In [69]:
from transformers import AutoTokenizer 
from datasets import load_dataset, load_metric

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, device='gpu')

In [70]:
example = ner_train[5]
print(example["tokens"])

['<', 'НАЗВАНИЕ', ':', '>', 'КВН', '2021', 'Высшая', 'лига', '=', 'Ситуация', ':', 'Бузова', ',', 'Меладзе', ',', 'Ревва', '.', 'Первая', '1/8', '<', 'ОПИСАНИЕ', ':', '>', 'YouTube', 'youtube', '.', 'com', '/', 'kvn', 'Официальный', 'сайт', 'КВН', 'kvn', '.', 'ru', '/', 'VK', 'vk', '.', 'com', '/', 'kvnofficial', 'Приобрести', 'билет', 'на', 'игры', 'Высшей', 'Лиги', 'КВН', '<', 'LINK', '>', '#', 'КВН', '#', 'квн', '2022', '#', 'шутки', '#', 'юмор', '#', 'смешно', '#', 'михеева', '#', 'ревва', '#', 'бузова', '#', 'меладзе', '#', 'эрнст', '#', 'москва', '#', 'высшаялига', '#', 'высшаялигаКВН']


In [71]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', '<', 'Н', '##АЗ', '##В', '##АН', '##И', '##Е', ':', '>', 'К', '##В', '##Н', '2021', 'Вы', '##сшая', 'лига', '=', 'С', '##ит', '##уа', '##ция', ':', 'Бу', '##зова', ',', 'М', '##ела', '##дзе', ',', 'Р', '##ев', '##ва', '.', 'Первая', '1', '/', '8', '<', 'О', '##П', '##И', '##С', '##АН', '##И', '##Е', ':', '>', 'YouTube', 'youtube', '.', 'com', '/', 'k', '##vn', 'Официальный', 'сайт', 'К', '##В', '##Н', 'k', '##vn', '.', 'ru', '/', 'V', '##K', 'v', '##k', '.', 'com', '/', 'k', '##vn', '##off', '##icia', '##l', 'При', '##об', '##рест', '##и', 'биле', '##т', 'на', 'игры', 'Вы', '##с', '##шей', 'Лиги', 'К', '##В', '##Н', '<', 'L', '##IN', '##K', '>', '#', 'К', '##В', '##Н', '#', 'кв', '##н', '2022', '#', 'ш', '##ут', '##ки', '#', 'юм', '##ор', '#', 'см', '##еш', '##но', '#', 'ми', '##хе', '##ева', '#', 'р', '##ев', '##ва', '#', 'бу', '##зова', '#', 'м', '##ела', '##дзе', '#', 'э', '##рнст', '#', 'м', '##ос', '##ква', '#', 'вы', '##сшая', '##ли', '##га', '#', 'вы', '##сшая', '##ли'

In [72]:
len(example["tags"]), len(tokenized_input["input_ids"])

(79, 152)

In [73]:
print(tokenized_input.word_ids())

[None, 0, 1, 1, 1, 1, 1, 1, 2, 3, 4, 4, 4, 5, 6, 6, 7, 8, 9, 9, 9, 9, 10, 11, 11, 12, 13, 13, 13, 14, 15, 15, 15, 16, 17, 18, 18, 18, 19, 20, 20, 20, 20, 20, 20, 20, 21, 22, 23, 24, 25, 26, 27, 28, 28, 29, 30, 31, 31, 31, 32, 32, 33, 34, 35, 36, 36, 37, 37, 38, 39, 40, 41, 41, 41, 41, 41, 42, 42, 42, 42, 43, 43, 44, 45, 46, 46, 46, 47, 48, 48, 48, 49, 50, 50, 50, 51, 52, 53, 53, 53, 54, 55, 55, 56, 57, 58, 58, 58, 59, 60, 60, 61, 62, 62, 62, 63, 64, 64, 64, 65, 66, 66, 66, 67, 68, 68, 69, 70, 70, 70, 71, 72, 72, 73, 74, 74, 74, 75, 76, 76, 76, 76, 77, 78, 78, 78, 78, 78, 78, 78, None]


In [74]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example["tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

152 152


#### У Bert свой собсвенный токенайзер, который разбивает слова на мелкие токены, поэтому нам нужно корректно сопоставить токены и соответсвующие им неры.

In [75]:
def tokenize_and_align_labels(examples, label_all_tokens=True):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples['tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        label_ids = [label_list.index(idx) if isinstance(idx, str) else idx for idx in label_ids]

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [76]:
tokenize_and_align_labels(ner_data['train'][1:2])

{'input_ids': [[2, 32, 293, 16218, 8117, 20795, 8759, 14386, 30, 34, 282, 16474, 28649, 18397, 866, 12515, 28941, 603, 1154, 11775, 320, 12383, 16615, 23912, 16746, 25993, 17968, 32, 294, 3932, 8759, 3330, 20795, 8759, 14386, 30, 34, 1, 282, 1647, 1736, 650, 2448, 18564, 299, 27511, 6062, 25993, 17968, 23677, 314, 16474, 28649, 22655, 603, 310, 24464, 2395, 2462, 16, 18397, 866, 12515, 28941, 603, 16, 14698, 1928, 1154, 11775, 1, 1041, 11273, 18397, 866, 12515, 28941, 603, 20377, 15952, 914, 5911, 16, 619, 15158, 16, 1294, 14860, 18, 290, 4829, 12515, 28941, 29, 4495, 27369, 2629, 314, 18556, 2841, 18397, 866, 27804, 16, 28195, 14063, 2182, 548, 23897, 314, 18397, 866, 864, 9767, 2225, 11011, 8809, 11513, 20377, 721, 1747, 15158, 748, 14860, 18, 1, 297, 23949, 12250, 4159, 1186, 1690, 19634, 329, 18397, 866, 12515, 28941, 761, 1, 1, 11327, 303, 11806, 2412, 16, 17889, 284, 22626, 603, 16, 21, 13404, 300, 4845, 613, 3330, 4843, 31, 16632, 4829, 9504, 3130, 3074, 13650, 2535, 16, 12383, 

In [77]:
tokenized_datasets = ner_data.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/5779 [00:00<?, ? examples/s]

Map:   0%|          | 0/643 [00:00<?, ? examples/s]

#### Сохраняем словарик соотвествия тега и его индекса внутри модели

In [78]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))
model.config.id2label = dict(enumerate(label_list))
model.config.label2id = {v: k for k, v in model.config.id2label.items()}

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [79]:
# Специальный объект для удобного формирования батчей
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

### В качестве метрик возьмем precision, recall, accuracy, для этого воспользуемся специализированной под Ner задачу библиотеку seqeval

In [80]:
metric = load_metric("seqeval")

In [81]:
example = ner_train[4]
labels = example['tags']
metric.compute(predictions=[labels], references=[labels])

{'локация': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'название проекта': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'сезон': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'серия': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [82]:
args = TrainingArguments(
    "ner",
    evaluation_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
)

In [83]:
# что мы видим без дообучения модели
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels, zero_division=0)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.evaluate()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 3.253539800643921,
 'eval_precision': 0.003507906492367563,
 'eval_recall': 0.04118404118404118,
 'eval_f1': 0.006465136247695532,
 'eval_accuracy': 0.046201694702392826,
 'eval_runtime': 3.36,
 'eval_samples_per_second': 191.368,
 'eval_steps_per_second': 6.25}

In [84]:
import logging
from transformers.trainer import logger as noisy_logger
noisy_logger.setLevel(logging.WARNING)

In [85]:
# Для дообучения берта можно эксперементировать с заморозкой/разморозкой разных слоев, здесь мы оставим все слои размороженными 
# Для быстроты обучения можно заморозить всю бертовую часть, кроме классификатора, но тогда качесвто будет похуже
for param in model.parameters():
    param.requires_grad = True

In [86]:
args = TrainingArguments(
    "ner",
    evaluation_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
)

In [87]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.742367,0.000000,0.000000,0.000000,0.862791
2,No log,0.634384,0.101266,0.008580,0.015820,0.865133
3,0.894800,0.604962,0.138696,0.021450,0.037154,0.869240
4,0.894800,0.582472,0.182393,0.037109,0.061670,0.870526
5,0.894800,0.565660,0.233480,0.056843,0.091427,0.872373
6,0.560900,0.551092,0.244702,0.065637,0.103510,0.873577
7,0.560900,0.539090,0.246538,0.078293,0.118844,0.874759


In [32]:
trainer.evaluate()

{'eval_loss': 0.5137659311294556,
 'eval_precision': 0.40093277581215825,
 'eval_recall': 0.3872320596458527,
 'eval_f1': 0.39396333754740837,
 'eval_accuracy': 0.8871988967516584,
 'eval_runtime': 3.6317,
 'eval_samples_per_second': 177.05,
 'eval_steps_per_second': 5.782,
 'epoch': 100.0}

In [33]:
# Посчитаем метрики на отложенном датасете

predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'Дата': {'precision': 0.5478119935170178,
  'recall': 0.6068222621184919,
  'f1': 0.5758091993185689,
  'number': 557},
 'бренд': {'precision': 0.34104046242774566,
  'recall': 0.2062937062937063,
  'f1': 0.2570806100217865,
  'number': 286},
 'вид спорта': {'precision': 0.0625,
  'recall': 0.011764705882352941,
  'f1': 0.019801980198019802,
  'number': 85},
 'видеоигра': {'precision': 0.26885245901639343,
  'recall': 0.26198083067092653,
  'f1': 0.26537216828478966,
  'number': 313},
 'команда': {'precision': 0.34513274336283184,
  'recall': 0.38235294117647056,
  'f1': 0.3627906976744186,
  'number': 102},
 'лига': {'precision': 0.16393442622950818,
  'recall': 0.2127659574468085,
  'f1': 0.18518518518518517,
  'number': 47},
 'локация': {'precision': 0.4199134199134199,
  'recall': 0.4835493519441675,
  'f1': 0.4494902687673772,
  'number': 1003},
 'модель': {'precision': 0.25925925925925924,
  'recall': 0.24873096446700507,
  'f1': 0.2538860103626943,
  'number': 197},
 'название 

In [34]:
from sklearn.metrics import confusion_matrix
import pandas as pd

In [35]:
cm = pd.DataFrame(
    confusion_matrix(sum(true_labels, []), sum(true_predictions, []), labels=label_list),
    index=label_list,
    columns=label_list
)
cm

,O,B-Дата,B-бренд,B-вид спорта,B-видеоигра,B-команда,B-лига,B-локация,B-модель,B-название проекта,...,I-видеоигра,I-команда,I-лига,I-локация,I-модель,I-название проекта,I-организация,I-персона,I-сезон,I-серия
O,82603,154,55,12,90,38,15,331,25,404,...,163,4,43,188,122,608,173,408,0,1
B-Дата,138,387,0,0,0,0,0,1,0,1,...,0,0,0,0,0,3,0,0,0,0
B-бренд,150,0,64,0,7,0,0,7,21,20,...,2,0,0,3,0,3,0,0,0,0
B-вид спорта,76,0,0,2,0,0,3,0,0,2,...,0,0,2,0,0,0,0,0,0,0
B-видеоигра,175,0,0,0,99,0,0,0,2,12,...,15,0,0,0,0,6,1,0,0,0
B-команда,27,0,0,0,0,42,0,0,0,6,...,0,0,0,0,0,0,0,5,0,0
B-лига,28,0,0,0,0,1,7,0,0,5,...,0,0,6,0,0,0,0,0,0,0
B-локация,319,2,2,0,0,5,0,554,0,8,...,0,0,2,55,0,12,4,5,0,0
B-модель,86,0,23,0,4,0,0,0,63,5,...,5,0,0,0,10,0,0,0,0,0
B-название проекта,658,0,3,0,14,4,0,3,0,414,...,5,0,1,0,0,34,2,7,0,0


In [36]:
model.save_pretrained(name_folder_and_model)
tokenizer.save_pretrained(name_folder_and_model)

('ner_bert_35.bin\\tokenizer_config.json',
 'ner_bert_35.bin\\special_tokens_map.json',
 'ner_bert_35.bin\\vocab.txt',
 'ner_bert_35.bin\\added_tokens.json',
 'ner_bert_35.bin\\tokenizer.json')

### Посмотрим на результаты

In [50]:
# text = ' '.join(ner_train[25]['tokens'])
text = ner_train[25]['tokens']

In [51]:
import torch
from transformers import pipeline
import re
pipe = pipeline(model=model, tokenizer=tokenizer, task='ner', aggregation_strategy='average', device='cpu')

def predict_ner(text, tokenizer, model, pipe, verbose=True):

    dismissed_token = re.compile(r'\xad+|\u200b+')
    text = [re.sub(dismissed_token, '[UNK]', tok) for tok in text]

    tokens = tokenizer(text, truncation=True, is_split_into_words=True, return_tensors='pt')
    words = tokens.words()

    tokens = {k: v.to(model.device) for k, v in tokens.items()}

    with torch.no_grad():
        pred = model(**tokens)

    indices = pred.logits.argmax(dim=-1)[0].cpu().numpy()
    labels = []
    prev=words[1] # это всегда ноль - первое слово
    labels = [label_list[indices[1]]]
    for word, tag in zip(words[1:-1], indices[1:-1]):
        if word != prev:
            labels.append(label_list[tag])
            prev=word
    return labels

In [52]:
predict_ner(text, tokenizer, model, pipe)

C:\Users\tommy\projects\2023-10-hackaton-rutube\dataset\venv\Lib\site-packages\transformers\tokenization_utils_base.py:360: FutureWarning: `BatchEncoding.words()` property is deprecated and should be replaced with the identical, but more self-explanatory `BatchEncoding.word_ids()` property.
  warnings.warn(


['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-персона',
 'I-персона',
 'O',
 'B-название проекта',
 'I-название проекта',
 'I-название проекта',
 'O',
 'O',
 'B-Дата',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-персона',
 'I-персона',
 'I-персона',
 'O',
 'O',
 'O',
 'B-организация',
 'I-организация',
 'I-организация',
 'O',
 'O',
 'O',
 'O',
 'B-Дата',
 'B-Дата',
 'O',
 'O',
 'O',
 'O',
 'B-Дата',
 'O',
 'O',
 'O',
 'B-организация',
 'I-организация',
 'I-организация',
 'I-организация',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-организация',
 'I-организация',
 'I-организация',
 'I-организация',
 'O',
 'I-организация',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-организация',
 'I-организация',
 'I-организация',
 'I-организация',
 'I-организация',
 'O',
 'O',
 'O',
 'O',
 'I-организация',
 'I-организация',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'I-организация',
 'I-организация',
 'O',
 'O',
 'O',
 'B-Дата',
 'O',
 'O',
 'O']

### Тестового датасета у вас пока нет, по которому будет считаться метрика на лидерборде, но прогоним для примера через нашу отложенную выборку, чтобы понять формат выходных данных.
ВАЖНО: в тестовом датасете у вас будет тест в том же формате, что он был в трейне 'video_info', в финальном сабмишене эту колонку и индексы менять нельзя, нужно будет только заполнить колонку 'entities_prediction'

In [53]:
#from tqdm.notebook import tqdm

submission = pd.DataFrame(columns=[['video_info', 'entities_prediction']])
submission['entities_prediction'] = submission['entities_prediction'].astype('object')
def sample_submission(tokenizer, model, pipe, submission):

    for i, elem in dft.iterrows():
        tokens = extract_labels_test(elem)
        labels = predict_ner(tokens['tokens'], tokenizer, model, pipe, verbose=False)
        if len(tokens['tokens'])!= len(labels):
            print(tokens['tokens'])
        submission.loc[i, 'video_info'] = elem['video_info']

        submission.loc[i, 'entities_prediction'] = [[label] for label in labels]
    return submission

In [54]:
result = sample_submission( tokenizer, model, pipe, submission)

C:\Users\tommy\projects\2023-10-hackaton-rutube\dataset\venv\Lib\site-packages\transformers\tokenization_utils_base.py:360: FutureWarning: `BatchEncoding.words()` property is deprecated and should be replaced with the identical, but more self-explanatory `BatchEncoding.word_ids()` property.
  warnings.warn(


In [55]:
result

,video_info,entities_prediction
0,"<НАЗВАНИЕ:> БОЕЦ БЕЗ ПРАВИЛ, ТРЕЙЛЕР на русском, фильм 2021| боевик, MMA <ОПИСАНИЕ:>Больше информации в нашей группе ВК: <LINK> Русский трейлер фильма БОЕЦ БЕЗ ПРАВИЛ 2021 г 🎬 Боец без правил (2021) русский трейлер Notorious Nick (2021)/ США драма, боевик Дата выхода: 7 августа 2021 г (мир) 2 се...","[O, O, O, O, O, O, O, O, O, O, O, O, O, B-Дата, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-Дата, I-Дата, O, O, O, O, O, B-Дата, O, O, O, O, O, O, B-Дата, O, O, B-локация, O, O, O, O, O, O, B-Дата, I-Дата, I-Дата, I-Дата, O, O, O, B-Дата, I-Дата, I-Дата, I-Дата, O, ..."
1,"<НАЗВАНИЕ:> ШОК! КАК ЖЕ ЭТОМУ БОМЖУ ВЕЗЕТ В Brawl Stars ОЧЕРЕДНОЕ ОТКРЫТИЕ СУНДУКОВ <ОПИСАНИЕ:>Проект путь Бомжа в Бравл Старс, смотрите как же везет на этом Аккаунте, открыл сундук и выпал новый бравлер Brawl Stars Крутые ПЛЮШКИ: <LINK> Для коммерческих запросов: <AT> Для личных обращений: <AT>...","[O, O, O, O, O, O, O, O, O, O, O, O, I-видеоигра, I-видеоигра, O, O, O, O, O, O, O, O, O, O, O, B-видеоигра, I-видеоигра, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-видеоигра, I-видеоигра, O, O, O, O, O, O, O, O, O, O, O..."
2,<НАЗВАНИЕ:> ДРЕВНИЕ РОБОТЫ NATASHA TALON И LEO CLAW ПРОТИВ АМЕРИКАНСКОГО КЛАНА RINO MK3 В ГОЛОСЕ WAR ROBOTS 2022 <ОПИСАНИЕ:>Играть В WAR ROBOTS: <LINK> ТЕЛЕГРАМ WR COMMANDER <LINK> подписывайтесь чтобы не потеряться ! ПОДДЕРЖАТЬ КАНАЛ АВТОРА: я благодарен вашей поддержке всем добра и мира: donat...,"[O, O, O, O, O, O, B-видеоигра, I-видеоигра, O, B-видеоигра, I-видеоигра, O, O, O, I-видеоигра, I-видеоигра, I-видеоигра, O, O, I-видеоигра, I-видеоигра, B-Дата, O, O, O, O, O, O, B-видеоигра, I-видеоигра, O, O, O, O, B-организация, I-видеоигра, I-видеоигра, O, O, O, O, O, O, O, O, O, O, O, O, O..."
3,"<НАЗВАНИЕ:> ЖЕНА ПУТЕШЕСТВЕННИКА ВО ВРЕМЕНИ = СМОТРИМ ОБЗОР ВМЕСТЕ <ОПИСАНИЕ:> Если понравилось ставьте ракетку 🚀 Если смотрел(а) напиши комментарий: ""Как тебе?"" Трейлер тут: <LINK> Шоу ""Без слов"" <LINK>/ #шоуспашей #папаянчика #женапутешественникавовремени #обзорсериалов #сериалы #сериалсвысоки...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-название проекта, I-название проекта, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
4,"<НАЗВАНИЕ:> Кинозал ДК приглашает на мультфильм ""Потеряшки и Тайна волшебной пирамиды"", 6+, 80 мин. #кинозалДКям <ОПИСАНИЕ:>Кинозал ДК Гаврилов Ямского МР приглашает с 31 марта на мультфильм ""Потеряшки и Тайна волшебной пирамиды"", 6+, 80 мин. Стоимость билетов 100 220 руб. НЕ ЗАБУДЬТЕ ВЗЯТЬ С СО...","[O, O, O, O, O, I-организация, O, O, O, O, B-название проекта, I-название проекта, I-название проекта, I-название проекта, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, I-локация, I-локация, I-локация, I-локация, O, O, B-Дата, I-Дата, O, O, O, B-название проекта, O, O, O, O, O, O, O, O, O, O, ..."
...,...,...
1601,"<НАЗВАНИЕ:> Милейшие детеныши животных, которые заставят вас улыбнуться <ОПИСАНИЕ:>Милейшие детеныши животных, которые заставят вас улыбнуться Когда денек не задался и кажется, что ничего хорошего в вашей жизни не происходит, когда весь интернет завален удручающими новостями, когда начинает угас...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O..."
1602,"<НАЗВАНИЕ:> Диана Шурыгина набросилась на зрителей в студии: "".... Пусть говорят. Фрагмент выпуска от 21.02.2017 <ОПИСАНИЕ:>17 летняя Диана Шурыгина вступила в открытую конфронтацию с экспертами в студии, не выдержав критики в свой адрес. Успокоить героиню не смогли даже родители. Полный выпуск ...","[O, O, O, O, B-персона, I-персона, O, O, O, O, O, O, O, O, B-название проек

In [56]:
result.to_csv('submission2.csv', index=False)

In [ ]:
ner_test[0]